In [ ]:
!gdown --id '1lfeLzSgj09icKQT9Yrrvl3quFrwTN9bb' --output data.zip
!unzip data.zip

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git

In [3]:
import pathlib

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow_examples.models.pix2pix import pix2pix

width = 416
height = 416

def load_and_preprocess_image(path):
    img = tf.io.read_file(path + '/img.png')
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [width, height])

    label = tf.io.read_file(path + '/label.png')
    label = tf.image.decode_jpeg(label, channels=1)
    label = tf.image.resize(label, [width, height])
    label /= 255.0

    return img, label


def unet_model(output_channels):
    inputs = tf.keras.layers.Input(shape=[width, height, 3])
    x = inputs

    # 在模型中降频取样
    skips = down_stack(x)
    x = skips[-1]
    skips = reversed(skips[:-1])

    # 升频取样然后建立跳跃连接
    for up, skip in zip(up_stack, skips):
        x = up(x)
        concat = tf.keras.layers.Concatenate()
        x = concat([x, skip])

    # 这是模型的最后一层
    last = tf.keras.layers.Conv2DTranspose(
        output_channels, 3, strides=2,
        padding='same')  # 64x64 -> 128x128

    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)


def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    plt.figure(figsize=(16, 16))
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i + 1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()


def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]


def show_predictions(dataset=None, num=2):
    if dataset:
        for img, mask in dataset.take(num):
            pred_mask = model.predict(img)
            display([img[0], mask[0], create_mask(pred_mask)])
    else:
        display([sample_image, sample_mask, create_mask(model.predict(sample_image[tf.newaxis, ...]))])

In [4]:
batch_size = 32
output_channels = 1
all_image_paths = [str(path) for path in pathlib.Path('./dataset').glob('*/*')]
image_count = len(all_image_paths)
steps_per_epoch = tf.math.ceil(image_count / batch_size).numpy()

In [5]:
np.random.shuffle(all_image_paths)

valid_count = int((tf.math.floor(image_count / 10)).numpy())

valid_image_path = all_image_paths[0 : valid_count]

train_image_path = all_image_paths[valid_count : ]

In [ ]:
image_ds = (
    tf.data.Dataset.from_tensor_slices(train_image_path)
        .map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
)

train_dataset = (
    image_ds.cache()
        .shuffle(image_count)
        .repeat()
        .batch(batch_size)
        .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
)

for image, mask in image_ds.take(2):
    sample_image, sample_mask = image, mask

display([sample_image, sample_mask])

base_model = tf.keras.applications.MobileNetV2(input_shape=[width, height, 3], include_top=False)

# 使用这些层的激活设置
layer_names = [
    'block_1_expand_relu',  # 64x64
    'block_3_expand_relu',  # 32x32
    'block_6_expand_relu',  # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',  # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]

# 创建特征提取模型
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),  # 32x32 -> 64x64
]

model = unet_model(output_channels)

model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['accuracy']
)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
show_predictions()

model_history = model.fit(train_dataset, epochs=30, steps_per_epoch=steps_per_epoch)

show_predictions(train_dataset)

In [ ]:
valid_ds = (
  tf.data.Dataset.from_tensor_slices(valid_image_path)
    .map(load_and_preprocess_image, tf.data.experimental.AUTOTUNE)
    .cache()
    .batch(100)
    .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
)

model.evaluate(valid_ds, verbose=2)

In [ ]:
show_predictions(valid_ds, 1)

In [ ]:
import cv2

def locate(img_src, img_mask, name):
    contours, hierarchy = cv2.findContours(img_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not len(contours):  # contours1长度为0说明未检测到车牌
        print("未检测到车牌")
    else:
        flag = 0  # 默认flag为0，因为不一定有车牌区域
        for ii, cont in enumerate(contours):
            x, y, w, h = cv2.boundingRect(cont)  # 获取最小外接矩形
            img_cut_mask = img_mask[y:y + h, x:x + w]  # 将标签车牌区域截取出来
            if w > 15 and h > 15:
                rect = cv2.minAreaRect(cont)  # 针对坐标点获取带方向角的最小外接矩形，中心点坐标，宽高，旋转角度
                box = cv2.boxPoints(rect).astype(np.int32)  # 获取最小外接矩形四个顶点坐标
                cv2.drawContours(img_mask, contours, -1, (0, 0, 255), 2)
                cv2.drawContours(img_mask, [box], 0, (0, 255, 0), 2)

                cont = cont.reshape(-1, 2).tolist()
                # 由于转换矩阵的两组坐标位置需要一一对应，因此需要将最小外接矩形的坐标进行排序，最终排序为[左上，左下，右上，右下]
                box = sorted(box, key=lambda xy: xy[0])  # 先按照左右进行排序，分为左侧的坐标和右侧的坐标
                box_left, box_right = box[:2], box[2:]  # 此时box的前2个是左侧的坐标，后2个是右侧的坐标
                box_left = sorted(box_left, key=lambda x: x[1])  # 再按照上下即y进行排序，此时box_left中为左上和左下两个端点坐标
                box_right = sorted(box_right, key=lambda x: x[1])  # 此时box_right中为右上和右下两个端点坐标
                box = np.array(box_left + box_right)  # [左上，左下，右上，右下]

                x0, y0 = box[0][0], box[0][1]  # 这里的4个坐标即为最小外接矩形的四个坐标，接下来需获取平行(或不规则)四边形的坐标
                x1, y1 = box[1][0], box[1][1]
                x2, y2 = box[2][0], box[2][1]
                x3, y3 = box[3][0], box[3][1]

                def point_to_line_distance(X, Y):
                    if x2 - x0:
                        k_up = (y2 - y0) / (x2 - x0)  # 斜率不为无穷大
                        d_up = abs(k_up * X - Y + y2 - k_up * x2) / (k_up ** 2 + 1) ** 0.5
                    else:  # 斜率无穷大
                        d_up = abs(X - x2)
                    if x1 - x3:
                        k_down = (y1 - y3) / (x1 - x3)  # 斜率不为无穷大
                        d_down = abs(k_down * X - Y + y1 - k_down * x1) / (k_down ** 2 + 1) ** 0.5
                    else:  # 斜率无穷大
                        d_down = abs(X - x1)
                    return d_up, d_down

                d0, d1, d2, d3 = np.inf, np.inf, np.inf, np.inf
                l0, l1, l2, l3 = (x0, y0), (x1, y1), (x2, y2), (x3, y3)
                for each in cont:  # 计算cont中的坐标与矩形四个坐标的距离以及到上下两条直线的距离，对距离和进行权重的添加，成功选出四边形的4个顶点坐标
                    x, y = each[0], each[1]
                    dis0 = (x - x0) ** 2 + (y - y0) ** 2
                    dis1 = (x - x1) ** 2 + (y - y1) ** 2
                    dis2 = (x - x2) ** 2 + (y - y2) ** 2
                    dis3 = (x - x3) ** 2 + (y - y3) ** 2
                    d_up, d_down = point_to_line_distance(x, y)
                    weight = 0.975
                    if weight * d_up + (1 - weight) * dis0 < d0:
                        d0 = weight * d_up + (1 - weight) * dis0
                        l0 = (x, y)
                    if weight * d_down + (1 - weight) * dis1 < d1:
                        d1 = weight * d_down + (1 - weight) * dis1
                        l1 = (x, y)
                    if weight * d_up + (1 - weight) * dis2 < d2:
                        d2 = weight * d_up + (1 - weight) * dis2
                        l2 = (x, y)
                    if weight * d_down + (1 - weight) * dis3 < d3:
                        d3 = weight * d_down + (1 - weight) * dis3
                        l3 = (x, y)

                p0 = np.float32([l0, l1, l2, l3])  # 左上角，左下角，右上角，右下角，形成的新box顺序需和原box中的顺序对应，以进行转换矩阵的形成
                p1 = np.float32([(0, 0), (0, 80), (240, 0), (240, 80)])
                transform_mat = cv2.getPerspectiveTransform(p0, p1)  # 构成转换矩阵
                lic = cv2.warpPerspective(img_src, transform_mat, (240, 80))  # 进行车牌矫正

                if len(contours) == 1:  # 只有一个区域可以认为是车牌区域
                    flag += 1
                    print('saving ', save_path + name[0:7] + '.png')
                    tf.io.write_file('./plate.jpeg', tf.image.encode_jpeg(lic))

        if not flag:
            print("未检测到车牌区域或车牌区域过小")


save_path = ''

for images, masks in valid_ds.take(1):
  for img in images:
    img = tf.keras.preprocessing.image.array_to_img(img)
    i = np.asarray(img)
    break
  for mask in masks:
    mask = tf.keras.preprocessing.image.array_to_img(mask)
    m = np.asarray(mask)
    break

  locate(i, m, 'sdfsdfsdf')